In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [34]:
# Load the dataset
data = pd.read_csv('training_set_features.csv')
data.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [35]:
# Check for missing values
print(data.isnull().sum())

# Handle missing values, for simplicity, let's fill them with the mode of each column
data.fillna(data.mode().iloc[0], inplace=True)


respondent_id                      0
xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

In [36]:
# Encode categorical features using LabelEncoder
label_encoders = {}
categorical_columns = data.select_dtypes(include=['object']).columns

for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


In [37]:
data2=pd.read_csv("training_set_labels.csv")
data2.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [38]:
# Define the features and target variables
features = data.drop(['respondent_id',], axis=1)
target_xyz = data2['xyz_vaccine']
target_seasonal = data2['seasonal_vaccine']

# Split the data into training and testing sets
X_train, X_test, y_train_xyz, y_test_xyz = train_test_split(features, target_xyz, test_size=0.2, random_state=42)
X_train, X_test, y_train_seasonal, y_test_seasonal = train_test_split(features, target_seasonal, test_size=0.2, random_state=42)


In [39]:
# Initialize the model
model_xyz = RandomForestClassifier(random_state=42)
model_seasonal = RandomForestClassifier(random_state=42)

# Train the model
model_xyz.fit(X_train, y_train_xyz)
model_seasonal.fit(X_train, y_train_seasonal)


RandomForestClassifier(random_state=42)

In [40]:
# Make predictions
predictions_xyz = model_xyz.predict_proba(X_test)[:, 1]
predictions_seasonal = model_seasonal.predict_proba(X_test)[:, 1]

# Evaluate the model
roc_auc_xyz = roc_auc_score(y_test_xyz, predictions_xyz)
roc_auc_seasonal = roc_auc_score(y_test_seasonal, predictions_seasonal)

# Calculate the mean ROC AUC score
mean_roc_auc = np.mean([roc_auc_xyz, roc_auc_seasonal])

print(f'ROC AUC Score for xyz_vaccine: {roc_auc_xyz}')
print(f'ROC AUC Score for seasonal_vaccine: {roc_auc_seasonal}')
print(f'Mean ROC AUC Score: {mean_roc_auc}')


ROC AUC Score for xyz_vaccine: 0.8285427644572189
ROC AUC Score for seasonal_vaccine: 0.8524193529039108
Mean ROC AUC Score: 0.8404810586805649


In [44]:
data3=pd.read_csv("test_set_features.csv")

data3.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [45]:
# Check for missing values
print(data3.isnull().sum())

# Handle missing values, for simplicity, let's fill them with the mode of each column
data3.fillna(data3.mode().iloc[0], inplace=True)


respondent_id                      0
xyz_concern                       85
xyz_knowledge                    122
behavioral_antiviral_meds         79
behavioral_avoidance             213
behavioral_face_mask              19
behavioral_wash_hands             40
behavioral_large_gatherings       72
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            932
child_under_6_months             813
health_worker                    789
health_insurance               12228
opinion_xyz_vacc_effective       398
opinion_xyz_risk                 380
opinion_xyz_sick_from_vacc       375
opinion_seas_vacc_effective      452
opinion_seas_risk                499
opinion_seas_sick_from_vacc      521
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4497
m

In [46]:
# Encode categorical features using LabelEncoder
label_encoders = {}
categorical_columns = data3.select_dtypes(include=['object']).columns

for column in categorical_columns:
    le = LabelEncoder()
    data3[column] = le.fit_transform(data3[column])
    label_encoders[column] = le


In [47]:
data3.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1,1,1,0,7,0,1.0,0.0,1,7
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,1,1,0,1,2,3.0,0.0,1,20
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1,0,0,0,5,2,1.0,0.0,10,12
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,5,0,1.0,0.0,4,21
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0,1,0,0,6,2,0.0,1.0,4,10


In [50]:
data3=data3.drop("respondent_id",axis=1)

In [51]:
data3=data3.drop(index=2)
data3.shape

(26707, 35)

In [22]:
data4=pd.read_csv("submission_format.csv")
data4.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.5,0.7
1,26708,0.5,0.7
2,26709,0.5,0.7
3,26710,0.5,0.7
4,26711,0.5,0.7


In [52]:
# Make predictions on the entire dataset for submission
all_predictions_xyz = model_xyz.predict_proba(data3)[:, 1]
all_predictions_seasonal = model_seasonal.predict_proba(data3)[:, 1]

# Create the submission dataframe
submission = pd.DataFrame({
    'respondent_id': data['respondent_id'],
    'xyz_vaccine': all_predictions_xyz,
    'seasonal_vaccine': all_predictions_seasonal
})

# Save the submission file
submission.to_csv('submission.csv', index=False)
